<a href="https://colab.research.google.com/github/Wannayli/MIDTERM/blob/main/Spark_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check the environment

In [1]:
!java --version
!python --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.9.16


In [2]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=dc3112a1b02872ff9094061c73e2efcc59b7c93685d52418d4cf5da001c025f6
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


Create the Spark session

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [18]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
streaming_df = spark.readStream\
    .format("json") \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "data/archive/device_data/") \
    .option("maxFilesPerTrigger", 1) \
    .load("sample_data/news_data")

Place the json file

In [29]:

# Node.js program to demonstrate the 
# fs.createWriteStream() method 
  
// Include fs module 
let fs = require('fs'), 
fs.createWriteStream( path, options )

writeStream.start();
FileSource[sample_data/news_data]

# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
streaming_df.show(truncate=False)

SyntaxError: ignored

In [ ]:
# Lets explode the data as devices contains list/array of device reading
from pyspark.sql.functions import explode, col

exploded_df = streaming_df \
    .select("customerId", "eventId", "eventOffset", "eventPublisher", "eventTime", "data") \
    .withColumn("devices", explode("data.devices")) \
    .drop("data")


Flattening dataframe

In [ ]:
# Flatten the exploded df
flattened_df = exploded_df \
    .selectExpr("link", "headline", "category", "short_description", "authors", 
                "date") 

Write streaming data to a output folder

In [ ]:
# Write the output to console sink to check the output
writing_df = flattened_df.writeStream \
    .format("json") \
    .option("path", "data/output-5999222020/device_data") \
    .option("checkpointLocation","checkpoint_dir") \
    .outputMode("append") \
    .start()
    
# Start the streaming application to run until the following happens
# 1. Exception in the running program
# 2. Manual Interruption
writing_df.awaitTermination()

Check the output

In [ ]:
# Check the data at the output location

out_df = spark.read.json("data/output-5999222020/news_data/")
out_df.show(truncate=False)